### Name :  1-1-CisPdf2CsvByIndiv :
### Function :  Extract text content from CIS PDF files 
### Source Data :  高雄市議會1屆1次至2屆8次市政總質詢

In [18]:
# -*- coding: utf-8 -*-
"""
Created on May 13 2020
@author: johnson
"""

# Extract PDF by Page
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
import io
import glob
import csv
import os.path
import pandas as pd

###  user function : extract text data from PDF file via pdfminer 

In [19]:
def extract_text_by_page(pdfFile):
    with open(pdfFile, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            resource_manager = PDFResourceManager()
            #codec = 'utf-8'
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            #  via TextConvert or HTMLConverter or XMLConverter
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            # print(text)
            yield text
            # close open handles
            converter.close()
            fake_file_handle.close()

### user function : 
###        1) receive PDF file
###        2) pass to  user function extract_text_by_page()   
###        3) write into output file

In [20]:
def export_as_csv(pdf_file, csv_path, isNew):
    global FSeqNo
    if isNew:
        opMode = "w"
    else:
        opMode = 'a'
    with open(csv_path, opMode, newline='', encoding='utf-8-sig') as csv_file:
        writer = csv.writer(csv_file)
        
        print("PDF File: %s" % pdf_file)
        if isNew:
            writer.writerow(['FileNo', 'FileName', 'Text'])   # write fields name into CSV file head
            
        pdfName = os.path.basename(pdf_file)
        FSeqNo += 1
        contain = ""
        for page in extract_text_by_page(pdf_file):
            contain = contain + page 
            
        writer.writerow([FSeqNo, pdfName, contain])

### main 

In [21]:
pdf_path = "/Users/wombat/Desktop/課程筆記/政治與資訊/NSYSU1082（莊老師）/pdf_test" 
pdfs = glob.glob("{}/*.pdf".format(pdf_path))
newFlag = True
FSeqNo = 0
csv_path = "politicaltest.csv"
for pdf in pdfs:
    export_as_csv(pdf, csv_path, newFlag)
    newFlag = False

PDF File: /Users/wombat/Desktop/課程筆記/政治與資訊/NSYSU1082（莊老師）/pdf_test/慶富案.pdf


### 讀資料

In [22]:
df2 = pd.read_csv("politicaltest.csv",encoding='utf-8') #匯資料
df2

,FileNo,FileName,Text
0,1,慶富案.pdf,273 立法院公報 第106卷 第89期 委員會紀錄 立法院第9屆第4會期外交及國防委員...
